In [4]:
print("###### Starting Recommendation engine ###### ")

###### Starting Recommendation engine ###### 


# CF - Matrix Factorization (SVD)

Giving the problem statement, after researching several methods The Matrix factorization SVD (Single Value Decomposition) was the most effective for this particular purpose.
* Decent Performance
* Easy/Fast creating
* Leverage the data (With quality) available

In [5]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rodrigo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Reading Prepared file from Data Engineer
df = pd.read_csv('book-recommend-data/dataframe.csv')

In [7]:
df.shape

(141081, 9)

In [8]:
###### Data Manipulation:Impression
impression_w = {
   'dislike': 0.1,
   'interact': 1.0,
   'view': 2.0, 
   'like': 3.0, 
   'add to cart': 4.0,
   'checkout': 5.0
}
df['impression'] = df['impression'].apply(lambda x: impression_w[x])

In [9]:
# Normalizing grouped data
def smooth_user_preference(x):
    return math.log(1+x, 2)    
df_full = df.groupby(['user', 'bookISBN'])['impression'].sum().apply(smooth_user_preference).reset_index()
df_full.head(10)

,user,bookISBN,impression
0,8,0002005018,1.584963
1,99,0312252617,2.321928
2,99,0312261594,2.321928
3,99,0316748641,2.000000
4,99,0446677450,2.584963
5,99,0451166892,0.137504
6,114,0446608653,2.584963
7,114,0446612545,2.584963
8,114,0446612618,2.321928
9,114,0451208080,2.321928


In [10]:
df.dtypes

user                   int64
bookISBN              object
impression           float64
location              object
age                  float64
bookName              object
author                object
yearOfPublication    float64
publisher             object
dtype: object

In [11]:
df[df.bookISBN == '3404139178'] # Single Example 3404139178

,user,bookISBN,impression,location,age,bookName,author,yearOfPublication,publisher
25,276866,3404139178,5.0,-1,35.0,-1,-1,-1.0,-1
35193,66483,3404139178,5.0,-1,35.0,-1,-1,-1.0,-1
45398,86583,3404139178,5.0,-1,35.0,-1,-1,-1.0,-1
56893,106534,3404139178,1.0,-1,35.0,-1,-1,-1.0,-1
70123,132500,3404139178,5.0,-1,35.0,-1,-1,-1.0,-1
82425,157065,3404139178,5.0,-1,35.0,-1,-1,-1.0,-1


In [12]:
#Creating a sparse pivot table 
users_items_pivot_matrix_df = df.pivot(index='user',columns='bookISBN',values='impression').fillna(0)
users_items_pivot_matrix_df.head(10)

bookISBN,0002005018,0002251760,0002259001,0003300277,0006374921,0006480764,000648302X,0006485014,0006485200,0006493580,...,950491036X,9580464162,9722015184,9722020609,9726101794,9726106141,9726116902,9727722458,9770390107900,9871138148
user,,,,,,,,,,,,,,,,,,,,,
8,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Converting into Matrix
users_items_pivot_matrix = users_items_pivot_matrix_df.as_matrix()
users_items_pivot_matrix[:10]

C:\Users\Rodrigo\Anaconda2\envs\bonsai\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app


array([[2., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
users_ids = list(users_items_pivot_matrix_df.index)
users_ids[:10]

[8, 99, 114, 232, 242, 243, 244, 254, 300, 383]

In [15]:
#Performs matrix factorization of the original user item matrix
U, S, V = svds(users_items_pivot_matrix)

In [16]:
U.shape

(13030, 6)

In [17]:
V.shape

(6, 11234)

In [18]:
S = np.diag(S)
S.shape

(6, 6)

In [22]:
all_user_predicted_ratings = np.dot(np.dot(U, S), V) 
all_user_predicted_ratings

array([[ 9.12173700e-04, -3.91099396e-05, -9.37033084e-06, ...,
        -1.06017540e-05,  2.40513882e-05, -1.34487324e-05],
       [-3.40767616e-04,  2.36032426e-04,  1.53426857e-04, ...,
         9.22316234e-05, -6.73036711e-06,  9.47265355e-05],
       [-9.19785987e-05,  1.71477853e-03,  1.18315553e-03, ...,
         4.65478934e-04,  1.67039519e-05,  6.74842469e-04],
       ...,
       [ 2.54421234e-03,  9.86144441e-03,  1.98754169e-03, ...,
         3.26867976e-03,  7.13518288e-05,  4.71137316e-03],
       [ 3.20560889e-03,  2.44727877e-04,  4.88521175e-05, ...,
         1.71854543e-05,  8.49404399e-05,  6.05347822e-05],
       [-5.41696375e-04, -2.46043049e-04,  1.38181086e-04, ...,
        -1.91811184e-04, -1.08547872e-05, -2.30572999e-04]])

In [20]:
#Converting the reconstructed matrix back to a Pandas dataframe
cf_preds_df = pd.DataFrame(all_user_predicted_ratings, columns = users_items_pivot_matrix_df.columns, index=users_ids)
cf_preds_df.head(10)

bookISBN,0002005018,0002251760,0002259001,0003300277,0006374921,0006480764,000648302X,0006485014,0006485200,0006493580,...,950491036X,9580464162,9722015184,9722020609,9726101794,9726106141,9726116902,9727722458,9770390107900,9871138148
8,0.000912,-0.000039,-0.000009,-0.000014,0.000001,-0.000006,0.000881,-1.046214e-06,-0.000034,-0.000014,...,-0.000038,6.487435e-07,-0.000008,-1.084348e-06,3.728171e-08,-0.000006,0.000673,-1.060175e-05,0.000024,-0.000013
99,-0.000341,0.000236,0.000153,0.000127,0.000010,0.000077,-0.000408,1.654906e-05,0.000092,0.000166,...,0.000187,6.408556e-06,0.000115,1.067737e-05,9.696559e-06,0.000094,-0.000252,9.223162e-05,-0.000007,0.000095
114,-0.000092,0.001715,0.001183,0.000753,0.000082,0.000367,0.000469,1.281586e-04,0.001062,0.000862,...,0.001402,4.798663e-05,0.000592,6.256843e-05,5.496279e-05,0.000483,0.000056,4.654789e-04,0.000017,0.000675
232,-0.000118,0.000127,0.000041,0.000029,0.000004,0.000014,-0.000102,4.969951e-06,0.000001,0.000022,...,0.000078,3.409908e-06,0.000025,3.424282e-06,2.983612e-06,0.000021,-0.000069,3.954071e-05,-0.000003,0.000057
242,0.002319,0.000325,0.000120,0.000084,0.000015,0.000023,0.002391,1.601303e-05,-0.000038,0.000036,...,0.000251,1.212501e-05,0.000042,7.983903e-06,1.026974e-05,0.000039,0.001820,1.264014e-04,0.000063,0.000181
243,0.000808,0.004905,0.003968,0.002106,0.000294,0.001370,0.001058,4.024108e-04,0.003063,0.002950,...,0.002833,1.687870e-04,0.002265,2.077129e-04,1.765521e-04,0.001866,0.000072,9.197695e-04,0.000092,0.001482
244,0.000561,0.003933,0.001596,0.000999,0.000161,0.000678,-0.000152,1.836664e-04,-0.000702,0.000991,...,0.002028,1.360884e-04,0.001145,1.344988e-04,1.260875e-04,0.000984,0.000636,1.284688e-03,0.000036,0.001768
254,-0.008853,0.049052,0.003261,0.007595,0.000787,-0.000043,0.008261,9.211509e-04,-0.009803,-0.002800,...,0.038253,1.016412e-03,0.000449,7.880007e-04,7.947795e-04,0.000729,0.011441,1.995681e-02,-0.000377,0.027526
300,0.001005,0.000003,0.000015,0.000007,0.000002,-0.000004,0.001037,1.933384e-06,0.000028,0.000001,...,0.000020,9.515546e-07,-0.000006,-4.876589e-07,5.571549e-07,-0.000005,0.000762,9.782724e-07,0.000027,0.000005
383,0.003587,0.000338,-0.000052,0.000015,0.000008,-0.000048,0.003737,9.294543e-07,-0.000166,-0.000115,...,0.000280,9.519406e-06,-0.000071,1.457385e-07,4.844691e-06,-0.000052,0.002875,1.632045e-04,0.000092,0.000228


In [17]:
# Saving the output
cf_preds_df.to_csv('book-recommend-data/output.csv', sep = ',')

> Enhancements
* Improve Books' information details (73% Missing) in order to create a Content-Based model as a service to be used for unknown users (There are some API out there providing this kind of information)
* For the first contact, I'd suggest the most Popular books (Popularity model)
* Implementation using SPARK MLlib (ALS) - If the volume requires that.

> References
* https://surprise.readthedocs.io/en/stable/getting_started.html
* https://www.kaggle.com/gspmoreira/recommender-systems-in-python-101
* https://beckernick.github.io/matrix-factorization-recommender/
* https://www.datacamp.com/community/tutorials/recommender-systems-python
* https://github.com/rounakbanik/movies/blob/master/movies_recommender.ipynb
* https://kerpanic.wordpress.com/2018/03/26/a-gentle-guide-to-recommender-systems-with-surprise/
* http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
* https://towardsdatascience.com/how-to-build-a-simple-song-recommender-296fcbc8c85
* https://towardsdatascience.com/various-implementations-of-collaborative-filtering-100385c6dfe0
* https://github.com/groverpr/Machine-Learning/blob/master/notebooks/02_Collaborative_Filtering.ipynb
* https://medium.com/@tomar.ankur287/content-based-recommender-system-in-python-2e8e94b16b9e
* http://www.albertauyeung.com/post/python-matrix-factorization/
* https://www.youtube.com/watch?v=ZspR5PZemcs
* http://nicolas-hug.com/blog/matrix_facto_3
* http://langvillea.people.cofc.edu/DISSECTION-LAB/Emmie%27sLSI-SVDModule/p5module.html



In [ ]:
print("###### Finishing Recommendation engine ###### ")